In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_name = "distilbert-base-uncased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

question = "Where is the Eiffel Tower located?"
context = "The Eiffel Tower is located in Paris."

result = qa_pipeline(question=question, context=context)
print(result)

In [5]:
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm
import evaluate

# Load SQuAD v1.1 validation set
dataset = load_dataset("squad", split="validation")

# Load the QA model
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Evaluation metrics
squad_metric = evaluate.load("squad")

# Predict and collect results
predictions = []
references = []

for item in tqdm(dataset, desc="Evaluating"):
    context = item["context"]
    question = item["question"]
    answers = item["answers"]["text"]

    try:
        output = qa_pipeline(question=question, context=context)
        predicted_answer = output["answer"]
    except:
        predicted_answer = ""

    predictions.append({"id": item["id"], "prediction_text": predicted_answer})
    references.append({"id": item["id"], "answers": {"answer_start": item["answers"]["answer_start"],
                                                     "text": answers}})

# Compute metrics
results = squad_metric.compute(predictions=predictions, references=references)

print("\n🧪 Evaluation Results:")
print(f"✅ Exact Match (EM): {results['exact_match']:.2f}")
print(f"✅ F1 Score: {results['f1']:.2f}")


Device set to use cpu
Evaluating: 100%|█████████████████████████████████████████████████████████| 10570/10570 [3:13:20<00:00,  1.10s/it]



🧪 Evaluation Results:
✅ Exact Match (EM): 78.94
✅ F1 Score: 86.56


In [11]:
import pandas as pd

# Prepare data to save
preds_data = []
for i in range(len(dataset)):
    question = dataset[i]['question']
    context = dataset[i]['context']
    predicted_answer = predictions[i]['prediction_text']
    ground_truth_answers = dataset[i]['answers']['text']

    preds_data.append({
        "question": question,
        "context": context,
        "predicted_answer": predicted_answer,
        "ground_truth_answer": ground_truth_answers[0] if ground_truth_answers else ""
    })

# Save to CSV
df_preds = pd.DataFrame(preds_data)
df_preds.to_csv("squad_predictions.csv", index=False)

print("✅ Predictions saved to 'squad_predictions.csv'")


✅ Predictions saved to 'squad_predictions.csv'


In [17]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

while True:
    context = input("\n📜 Enter context (or type 'exit' to quit):\n")
    if context.lower() == 'exit':
        break
    question = input("\n Enter your question:\n")

    result = qa_pipeline(question=question, context=context)
    print(f"\n Answer: {result['answer']}\n")


Device set to use cpu



📜 Enter context (or type 'exit' to quit):
 ARE YOU GOOD

 Enter your question:
 WHAT ARE YOU



 Answer: YOU GOOD




📜 Enter context (or type 'exit' to quit):
 MY NAME IS RIMSHA

 Enter your question:
 WHAT IS MY NAME



 Answer: RIMSHA




📜 Enter context (or type 'exit' to quit):
 THERE ARE LOTS OF BEES IN ROOM. 

 Enter your question:
 WHAT ARE IN THE ROOM?



 Answer: BEES




📜 Enter context (or type 'exit' to quit):
 exit
